In [1187]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hitters/Hitters.csv


# 1. Gerekli kütüphanelerin kurulumu ve Veri setini anlama

In [1188]:
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
scaler = StandardScaler()
normalizer = preprocessing.Normalizer()

#Veri setini okuma
hitters_data = pd.read_csv("/kaggle/input/hitters/Hitters.csv")
df = hitters_data.copy()
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [1189]:
#Kısa bilgiler
df.describe().T

,count,mean,std,min,25%,50%,75%,max
AtBat,322.0,380.928571,153.404981,16.0,255.25,379.5,512.00,687.0
Hits,322.0,101.024845,46.454741,1.0,64.00,96.0,137.00,238.0
HmRun,322.0,10.770186,8.709037,0.0,4.00,8.0,16.00,40.0
Runs,322.0,50.909938,26.024095,0.0,30.25,48.0,69.00,130.0
RBI,322.0,48.027950,26.166895,0.0,28.00,44.0,64.75,121.0
Walks,322.0,38.742236,21.639327,0.0,22.00,35.0,53.00,105.0
Years,322.0,7.444099,4.926087,1.0,4.00,6.0,11.00,24.0
CAtBat,322.0,2648.683230,2324.205870,19.0,816.75,1928.0,3924.25,14053.0
CHits,322.0,717.571429,654.472627,4.0,209.00,508.0,1059.25,4256.0
CHmRun,322.0,69.490683,86.266061,0.0,14.00,37.5,90.00,548.0


In [1190]:
#Veri boyutu
df.shape

(322, 20)

In [1191]:
#Kaç eksik değer var?
df.isnull().sum()

AtBat         0
Hits          0
HmRun         0
Runs          0
RBI           0
Walks         0
Years         0
CAtBat        0
CHits         0
CHmRun        0
CRuns         0
CRBI          0
CWalks        0
League        0
Division      0
PutOuts       0
Assists       0
Errors        0
Salary       59
NewLeague     0
dtype: int64

In [1192]:
#Eksik verilere bakış
df[df["Salary"].isnull()].head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
15,183,39,3,20,15,11,3,201,42,3,20,16,11,A,W,118,0,0,NaN,A
18,407,104,6,57,43,65,12,5233,1478,100,643,658,653,A,W,912,88,9,NaN,A
22,22,10,1,4,2,1,6,84,26,2,9,9,3,A,W,812,84,11,NaN,A
30,313,84,9,42,30,39,17,6890,1833,224,1033,864,1087,A,W,127,221,7,NaN,A


# 2. Eksik verileri silerek ve herhangi başka bir analiz yapmadan alınacak sonuçlar

## 2.1.1 Tüm veri seti ile basit doğrusal regresyon

In [1193]:
#Eksik verileri siliyoruz.
df.dropna(inplace = True)
df.shape

(263, 20)

In [1194]:
#Veri setinde sadece CHits değişkenini kullanmak için seçiyoruz.
X = df[["CHits"]]
y = df[["Salary"]]
X = scaler.fit_transform(X)
reg_model = LinearRegression()
reg_model.fit(X, y)

LinearRegression()

In [1195]:
print("Formula: "+str(reg_model.intercept_)+" "+str(reg_model.coef_)+"x1")

Formula: [535.92588213] [[247.15214091]]x1


In [1196]:
y_pred = reg_model.predict(X)

In [1197]:
from sklearn.metrics import mean_squared_error
sonuc_tum_veri_slr = np.sqrt(mean_squared_error(y, y_pred))
sonuc_tum_veri_slr

376.3643027728653

In [1198]:
sonuc_tum_veri_cv_slr = np.sqrt(np.mean(-cross_val_score(reg_model, X, y, cv = 10, scoring = "neg_mean_squared_error")))
sonuc_tum_veri_cv_slr

383.4526516640398

## 2.1.2 Train ve test olarak ayırarak alınan basit doğrusal regresyon

In [1199]:
X = df[["CHits"]]
y = df[["Salary"]]
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=46)

In [1200]:
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)

LinearRegression()

In [1201]:
print("Formula: "+str(reg_model.intercept_)+" "+str(reg_model.coef_)+"x1")

Formula: [537.78313303] [[270.03230409]]x1


In [1202]:
y_pred = reg_model.predict(X_train)

In [1203]:
sonuc_train_veri_slr = np.sqrt(mean_squared_error(y_train, y_pred))
sonuc_train_veri_slr

348.96784372179457

In [1204]:
y_pred = reg_model.predict(X_test)

In [1205]:
sonuc_test_veri_slr = np.sqrt(mean_squared_error(y_test, y_pred))
sonuc_test_veri_slr

472.230050228946

In [1206]:
sonuc_tt_veri_cv_slr = np.sqrt(np.mean(-cross_val_score(reg_model, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error")))
sonuc_tt_veri_cv_slr

352.69590920059227

# 2.2.1 Tüm veri seti ile çoklu doğrusal regresyon

In [1207]:
#Kategorik değişkenleri 1-0 şeklinde ifade ederiz
df = pd.get_dummies(df, columns = ['League', 'Division', 'NewLeague'], drop_first = True)

In [1208]:
y = df["Salary"]
X = df.drop('Salary', axis=1)
X = scaler.fit_transform(X)

In [1209]:
reg_model = LinearRegression()
reg_model.fit(X, y)

LinearRegression()

In [1210]:
print(reg_model.intercept_)

535.9258821292775


In [1211]:
print(reg_model.coef_)

[-291.0945557   337.83047948   37.85383676  -60.57247861  -26.99498379
  135.07389695  -16.69335888 -391.03865466   86.68761664  -14.18172332
  480.74713477  260.68988581 -213.89225864   78.76129639   53.73248973
  -22.16086217   31.2487626   -58.41406355  -12.34881102]


In [1212]:
y_pred = reg_model.predict(X)

In [1213]:
sonuc_tum_veri_clr = np.sqrt(mean_squared_error(y, y_pred))
sonuc_tum_veri_clr

303.34447253531613

In [1214]:
sonuc_tum_veri_cv_clr = np.sqrt(np.mean(-cross_val_score(reg_model, X, y, cv = 10, scoring = "neg_mean_squared_error")))
sonuc_tum_veri_cv_clr

341.46597732981047

## 2.2.2 Train ve test olarak ayrılan coklu dogrusal regresyon

In [1215]:
y = df["Salary"]
X = df.drop('Salary', axis=1)
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=46)

In [1216]:
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)

LinearRegression()

In [1217]:
print(reg_model.intercept_)

534.5934048105765


In [1218]:
print(reg_model.coef_)

[-1.81989481e+02  2.33568966e+02  5.25917808e+01  2.07325675e+01
 -3.46308954e+01  4.81565534e+01 -1.14645801e-01 -9.31358594e+02
  1.11333493e+03  1.98450308e+02  1.27811909e+02 -2.35205399e+02
 -1.84619883e+01  9.16202977e+01  2.31451518e+01 -4.36054446e+00
  3.69787336e+01 -2.73605842e+01 -2.69655270e+01]


In [1219]:
y_pred = reg_model.predict(X_train)

In [1220]:
sonuc_train_veri_clr = np.sqrt(mean_squared_error(y_train, y_pred))
sonuc_train_veri_clr

261.36151857439336

In [1221]:
y_pred = reg_model.predict(X_test)

In [1222]:
sonuc_test_veri_clr = np.sqrt(mean_squared_error(y_test, y_pred))
sonuc_test_veri_clr

479.0715037370217

In [1223]:
sonuc_tt_veri_cv_clr = np.sqrt(np.mean(-cross_val_score(reg_model, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error")))
sonuc_tt_veri_cv_clr

307.22576059069496

## 2.3.1 Tüm veri ile Ridge Regresyon

In [1224]:
y = df["Salary"]
X = df.drop('Salary', axis=1)
X = scaler.fit_transform(X)

In [1225]:
reg_model = Ridge()
reg_model.fit(X, y)

Ridge()

In [1226]:
print(reg_model.intercept_)

535.9258821292775


In [1227]:
print(reg_model.coef_)

[-270.47549797  296.56190392   18.1638957   -29.40456794   -9.21637609
  124.44029854  -38.69207918 -225.35662735  126.90023608   39.08043983
  320.21040942  160.36986132 -184.49438733   78.60950469   47.44645692
  -23.76376773   31.01887447  -60.24628766  -13.67402282]


In [1228]:
y_pred = reg_model.predict(X)

In [1229]:
sonuc_tum_veri_ridge = np.sqrt(mean_squared_error(y, y_pred))
sonuc_tum_veri_ridge

304.1140381775488

In [1230]:
sonuc_tum_veri_cv_ridge = np.sqrt(np.mean(-cross_val_score(reg_model, X, y, cv = 10, scoring = "neg_mean_squared_error")))
sonuc_tum_veri_cv_ridge

338.45318355057594

## 2.3.2 Train ve test ayırarak Ridge Regresyon

In [1231]:
y = df["Salary"]
X = df.drop('Salary', axis=1)
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=46)

In [1232]:
reg_model = Ridge()
reg_model.fit(X_train, y_train)

Ridge()

In [1233]:
print(reg_model.intercept_)

534.3842751448135


In [1234]:
print(reg_model.coef_)

[-2.12250580e+02  2.94521009e+02  5.61152622e+01 -2.55351846e+00
 -4.37183213e+01  6.01153833e+01 -3.86917951e+01 -2.61823619e+02
  2.83960961e+02  7.01505168e+01  2.64726580e+02 -1.60888933e+01
 -8.13605795e+01  9.06581582e+01  8.41866713e+00  6.12518006e-02
  3.19909624e+01 -3.13095890e+01 -2.03624248e+01]


In [1235]:
y_pred = reg_model.predict(X_train)

In [1236]:
sonuc_train_veri_ridge = np.sqrt(mean_squared_error(y_train, y_pred))
sonuc_train_veri_ridge

264.5767595094233

In [1237]:
y_pred = reg_model.predict(X_test)

In [1238]:
sonuc_test_veri_ridge = np.sqrt(mean_squared_error(y_test, y_pred))
sonuc_test_veri_ridge

458.75144252220133

In [1239]:
sonuc_tt_veri_cv_ridge = np.sqrt(np.mean(-cross_val_score(reg_model, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error")))
sonuc_tt_veri_cv_ridge

301.3084504111169

## 2.4.1 Tüm veri seti ile Lasso Regresyon

In [1240]:
y = df["Salary"]
X = df.drop('Salary', axis=1)
X = scaler.fit_transform(X)

In [1241]:
reg_model =Lasso()
reg_model.fit(X, y)

Lasso()

In [1242]:
print(reg_model.intercept_)

535.9258821292775


In [1243]:
print(reg_model.coef_)

[-281.1172681   303.71277457   11.13050868  -25.23750713   -0.
  120.83597005  -35.04328737 -161.32198712    0.           14.27182293
  375.15175777  192.2154112  -190.2391271    78.6764498    41.88526703
  -18.83445093   23.20900807  -58.23442843   -4.94183895]


In [1244]:
y_pred = reg_model.predict(X)

In [1245]:
sonuc_tum_veri_lasso = np.sqrt(mean_squared_error(y, y_pred))
sonuc_tum_veri_lasso

304.3172941943879

In [1246]:
sonuc_tum_veri_cv_lasso = np.sqrt(np.mean(-cross_val_score(reg_model, X, y, cv = 10, scoring = "neg_mean_squared_error")))
sonuc_tum_veri_cv_lasso

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9820.372265059501, tolerance: 4637.712860074581
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 16226.579207904637, tolerance: 4976.191260952606
  positive)


338.6265472629963

## 2.4.2 Train ve test ayırarak Lasso Regresyon

In [1247]:
y = df["Salary"]
X = df.drop('Salary', axis=1)
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=46)

In [1248]:
reg_model =Lasso()
reg_model.fit(X_train, y_train)

Lasso()

In [1249]:
print(reg_model.intercept_)

534.1205196004398


In [1250]:
print(reg_model.coef_)

[-196.30842607  276.77508177   51.8781116     0.          -34.54898782
   51.79290628  -24.37765742 -423.94957962  402.3140429    58.93022571
  269.48957057   -0.          -57.578216     89.43835188    8.90737726
    0.           25.73766269  -28.96185078  -14.25654751]


In [1251]:
y_pred = reg_model.predict(X_train)

In [1252]:
sonuc_train_veri_lasso = np.sqrt(mean_squared_error(y_train, y_pred))
sonuc_train_veri_lasso

263.7056447668435

In [1253]:
y_pred = reg_model.predict(X_test)

In [1254]:
sonuc_test_veri_lasso = np.sqrt(mean_squared_error(y_test, y_pred))
sonuc_test_veri_lasso

461.2422858691612

In [1255]:
sonuc_tt_veri_cv_lasso = np.sqrt(np.mean(-cross_val_score(reg_model, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error")))
sonuc_tt_veri_cv_lasso

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 16513.21239562705, tolerance: 3651.8483906293914
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7964.692977372557, tolerance: 3684.426158113166
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4623.6704056710005, tolerance: 3724.140602819258
  positive)


301.12052492949084

## 2.5.1 Tüm veri seti ile ElasticNet Regresyon

In [1256]:
y = df["Salary"]
X = df.drop('Salary', axis=1)
X = scaler.fit_transform(X)

In [1257]:
reg_model =ElasticNet()
reg_model.fit(X, y)

ElasticNet()

In [1258]:
print(reg_model.intercept_)

535.9258821292775


In [1259]:
print(reg_model.coef_)

[  5.76548475  44.32773203   1.63340483  28.13065891  22.59245714
  38.47123143   1.55719543  25.61720464  41.11724494  36.25263909
  41.71364158  43.50751186   8.82399977  52.26616588   5.18046378
 -10.83125396  12.90217625 -44.6639689    3.56603582]


In [1260]:
y_pred = reg_model.predict(X)

In [1261]:
sonuc_tum_veri_enet = np.sqrt(mean_squared_error(y, y_pred))
sonuc_tum_veri_enet

326.5621957219224

In [1262]:
sonuc_tum_veri_cv_enet = np.sqrt(np.mean(-cross_val_score(reg_model, X, y, cv = 10, scoring = "neg_mean_squared_error")))
sonuc_tum_veri_cv_enet

343.0594626712361

## 2.5.2 Train ve test ayırarak ElasticNet Regresyon

In [1263]:
y = df["Salary"]
X = df.drop('Salary', axis=1)
X = scaler.fit_transform(X)
for i in range(len(X)):
    if X[i][13]<0:
        X[i][13]=0
    else:
        X[i][13]>0
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=46)

In [1264]:
reg_model =ElasticNet()
reg_model.fit(X_train, y_train)

ElasticNet()

In [1265]:
print(reg_model.intercept_)

512.1240018644132


In [1266]:
print(reg_model.coef_)

[ 19.12370899  53.32781988  16.14761539  39.22136788  17.8814037
  26.41733876   1.57390652  27.77519247  43.49211963  28.51440379
  45.02864774  34.02397544  16.25505321  61.00880524  -8.52207635
  -1.54432882  12.49332455 -28.35242757   1.97108514]


In [1267]:
y_pred = reg_model.predict(X_train)

In [1268]:
sonuc_train_veri_enet = np.sqrt(mean_squared_error(y_train, y_pred))
sonuc_train_veri_enet

287.38052357656414

In [1269]:
y_pred = reg_model.predict(X_test)

In [1270]:
sonuc_test_veri_enet = np.sqrt(mean_squared_error(y_test, y_pred))
sonuc_test_veri_enet

462.7585299516961

In [1271]:
sonuc_tt_veri_cv_enet = np.sqrt(np.mean(-cross_val_score(reg_model, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error")))
sonuc_tt_veri_cv_enet

302.31448303262977

In [1272]:
#İlk sonuçların çıktısı
basicsonuc_df = pd.DataFrame({"All Data Error":[sonuc_tum_veri_slr,sonuc_tum_veri_clr,sonuc_tum_veri_ridge,sonuc_tum_veri_lasso,sonuc_tum_veri_enet],
                              "All Data cvScore(K-Fold)":[sonuc_tum_veri_cv_slr,sonuc_tum_veri_cv_clr,sonuc_tum_veri_cv_ridge,sonuc_tum_veri_cv_lasso,sonuc_tum_veri_cv_enet],
                              "Train Error":[sonuc_train_veri_slr,sonuc_train_veri_clr,sonuc_train_veri_ridge,sonuc_train_veri_lasso,sonuc_train_veri_enet],
                              "Test Error":[sonuc_test_veri_slr,sonuc_test_veri_clr,sonuc_test_veri_ridge,sonuc_test_veri_lasso,sonuc_test_veri_enet],                
                              "Train cvScore(K-Fold)":[sonuc_tt_veri_cv_slr,sonuc_tt_veri_cv_clr,sonuc_tt_veri_cv_ridge,sonuc_tt_veri_cv_lasso,sonuc_tt_veri_cv_enet]})
basicsonuc_df.index= ["SLR", "CLR","RID","LAS","ENE"]
basicsonuc_df

,All Data Error,All Data cvScore(K-Fold),Train Error,Test Error,Train cvScore(K-Fold)
SLR,376.364303,383.452652,348.967844,472.230050,352.695909
CLR,303.344473,341.465977,261.361519,479.071504,307.225761
RID,304.114038,338.453184,264.576760,458.751443,301.308450
LAS,304.317294,338.626547,263.705645,461.242286,301.120525
ENE,326.562196,343.059463,287.380524,462.758530,302.314483


# 3. Veri Ön işleme adımları(Eksik veri, aykırı veri analizleri)

In [1273]:
df = hitters_data.copy()
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [1274]:
#Yeni değişkenler oluşturuyoruz
catbat=df["CAtBat"]/df["Years"]
chits=df["CHits"]/df["Years"]
chmrun=df["CHmRun"]/df["Years"]
cruns=df["CRuns"]/df["Years"]
crbi=df["CRBI"]/df["Years"]
cwalks=df["CWalks"]/df["Years"]
df_seckin = pd.DataFrame({"ortAtBat":catbat,"ortHits":chits,"ortHmRun":chmrun,"ortRuns":cruns,"ortRBI":crbi,"ortWalks":cwalks})
df = pd.concat([df, df_seckin], axis=1)
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,...,Assists,Errors,Salary,NewLeague,ortAtBat,ortHits,ortHmRun,ortRuns,ortRBI,ortWalks
0,293,66,1,30,29,14,1,293,66,1,...,33,20,NaN,A,293.000000,66.000000,1.000000,30.000000,29.000000,14.000000
1,315,81,7,24,38,39,14,3449,835,69,...,43,10,475.0,N,246.357143,59.642857,4.928571,22.928571,29.571429,26.785714
2,479,130,18,66,72,76,3,1624,457,63,...,82,14,480.0,A,541.333333,152.333333,21.000000,74.666667,88.666667,87.666667
3,496,141,20,65,78,37,11,5628,1575,225,...,11,3,500.0,N,511.636364,143.181818,20.454545,75.272727,76.181818,32.181818
4,321,87,10,39,42,30,2,396,101,12,...,40,4,91.5,N,198.000000,50.500000,6.000000,24.000000,23.000000,16.500000


In [1275]:
df.corr()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,...,PutOuts,Assists,Errors,Salary,ortAtBat,ortHits,ortHmRun,ortRuns,ortRBI,ortWalks
AtBat,1.000000,0.967939,0.592198,0.913060,0.820539,0.669845,0.047372,0.235526,0.252717,0.236659,...,0.317550,0.353824,0.352117,0.394771,0.682601,0.686120,0.449912,0.675838,0.605747,0.447566
Hits,0.967939,1.000000,0.562158,0.922187,0.811073,0.641211,0.044767,0.227565,0.255815,0.202712,...,0.310673,0.320455,0.310038,0.438675,0.669229,0.703749,0.414794,0.675328,0.598846,0.429144
HmRun,0.592198,0.562158,1.000000,0.650988,0.855122,0.481014,0.116318,0.221882,0.220627,0.493227,...,0.282923,-0.106329,0.039318,0.343028,0.417737,0.411294,0.816323,0.466523,0.673801,0.371388
Runs,0.913060,0.922187,0.650988,1.000000,0.798206,0.732213,0.004541,0.186497,0.204830,0.227913,...,0.279347,0.220567,0.240475,0.419859,0.633326,0.649075,0.477686,0.708969,0.580733,0.515092
RBI,0.820539,0.811073,0.855122,0.798206,1.000000,0.615997,0.146168,0.294688,0.308201,0.441771,...,0.343186,0.106591,0.193370,0.449457,0.578336,0.589899,0.703965,0.583649,0.750534,0.432697
Walks,0.669845,0.641211,0.481014,0.732213,0.615997,1.000000,0.136475,0.277175,0.280671,0.332473,...,0.299515,0.149656,0.129382,0.443867,0.521255,0.517367,0.454593,0.594891,0.535173,0.762129
Years,0.047372,0.044767,0.116318,0.004541,0.146168,0.136475,1.000000,0.920289,0.903631,0.726872,...,-0.004684,-0.080638,-0.162140,0.400657,0.367124,0.382881,0.264486,0.339320,0.398562,0.363858
CAtBat,0.235526,0.227565,0.221882,0.186497,0.294688,0.277175,0.920289,1.000000,0.995063,0.798836,...,0.062283,0.002038,-0.066922,0.526135,0.621840,0.634487,0.416974,0.592377,0.610182,0.546807
CHits,0.252717,0.255815,0.220627,0.204830,0.308201,0.280671,0.903631,0.995063,1.000000,0.783306,...,0.076547,-0.002523,-0.062756,0.548910,0.629950,0.655927,0.412098,0.608389,0.618941,0.540880
CHmRun,0.236659,0.202712,0.493227,0.227913,0.441771,0.332473,0.726872,0.798836,0.783306,1.000000,...,0.112724,-0.158511,-0.138115,0.524931,0.486955,0.484482,0.731355,0.508524,0.710430,0.521286


In [1276]:
df = df.drop(['AtBat','Hits','HmRun','Runs','RBI','Walks','Assists','Errors',"PutOuts",'League','NewLeague'],axis=1)

In [1277]:
df = pd.get_dummies(df, columns =["Division"], drop_first = True)

In [1278]:
df.head()

,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,Salary,ortAtBat,ortHits,ortHmRun,ortRuns,ortRBI,ortWalks,Division_W
0,1,293,66,1,30,29,14,NaN,293.000000,66.000000,1.000000,30.000000,29.000000,14.000000,0
1,14,3449,835,69,321,414,375,475.0,246.357143,59.642857,4.928571,22.928571,29.571429,26.785714,1
2,3,1624,457,63,224,266,263,480.0,541.333333,152.333333,21.000000,74.666667,88.666667,87.666667,1
3,11,5628,1575,225,828,838,354,500.0,511.636364,143.181818,20.454545,75.272727,76.181818,32.181818,0
4,2,396,101,12,48,46,33,91.5,198.000000,50.500000,6.000000,24.000000,23.000000,16.500000,0


In [1279]:
df.isnull().sum()

Years          0
CAtBat         0
CHits          0
CHmRun         0
CRuns          0
CRBI           0
CWalks         0
Salary        59
ortAtBat       0
ortHits        0
ortHmRun       0
ortRuns        0
ortRBI         0
ortWalks       0
Division_W     0
dtype: int64

In [1280]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors = 5)
df_filled = imputer.fit_transform(df)

In [1281]:
df = pd.DataFrame(df_filled,columns = df.columns)

In [1282]:
Q1 = df.Salary.quantile(0.25)
Q3 = df.Salary.quantile(0.75)
IQR = Q3-Q1
lower = Q1 - 1.5*IQR
upper = Q3 + 1.5*IQR
df.loc[df["Salary"] > upper,"Salary"] = upper

In [1283]:
from sklearn.neighbors import LocalOutlierFactor
lof =LocalOutlierFactor(n_neighbors= 20)
lof.fit_predict(df)

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,
        1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,
        1,  1,  1,  1,  1

In [1284]:
df_scores = lof.negative_outlier_factor_

In [1285]:
np.sort(df_scores)[0:30]

array([-6.56220735, -5.14766457, -4.35069729, -3.48856435, -3.47050448,
       -3.40846636, -3.39213388, -2.00187456, -1.9228802 , -1.71751942,
       -1.6841345 , -1.61831991, -1.60607412, -1.58545986, -1.50208784,
       -1.50066937, -1.49737285, -1.49652573, -1.40712201, -1.39826949,
       -1.39480135, -1.39149714, -1.3542251 , -1.33963683, -1.33529675,
       -1.3331065 , -1.32184747, -1.31779825, -1.3169861 , -1.31424581])

In [1286]:
th = np.sort(df_scores)[9]
th

-1.7175194228638802

In [1287]:
outlier = df_scores > th

In [1288]:
df = df[df_scores > th]

In [1289]:
df.shape

(312, 15)

# 4. Veri ön işleme yapıldıktan sonra alınan sonuçlar

## 4.1.1 Tüm veri ile basit doğrusal regresyon

In [1290]:
from sklearn.linear_model import LinearRegression
X = df[["ortHits"]]
y = df[["Salary"]]
X = scaler.fit_transform(X)
reg_model = LinearRegression()
reg_model.fit(X, y)

LinearRegression()

In [1291]:
print("Formula: "+str(reg_model.intercept_)+" "+str(reg_model.coef_)+"x1")

Formula: [503.38279359] [[248.74231967]]x1


In [1292]:
y_pred = reg_model.predict(X)

In [1293]:
from sklearn.metrics import mean_squared_error
sonuc_tum_veri_detay_slr = np.sqrt(mean_squared_error(y, y_pred))
sonuc_tum_veri_detay_slr

287.2045856967869

In [1294]:
sonuc_tum_veri_cv_detay_slr = np.sqrt(np.mean(-cross_val_score(reg_model, X, y, cv = 10, scoring = "neg_mean_squared_error")))
sonuc_tum_veri_cv_detay_slr

290.0577255449929

## 4.1.2 Train ve test ayırarak basit doğrusal regresyon

In [1295]:
X = df[["ortHits"]]
y = df[["Salary"]]
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=46)

In [1296]:
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)

LinearRegression()

In [1297]:
print("Formula: "+str(reg_model.intercept_)+" "+str(reg_model.coef_)+"x1")

Formula: [508.25579759] [[250.04368222]]x1


In [1298]:
y_pred = reg_model.predict(X_train)

In [1299]:
sonuc_train_veri_detay_slr = np.sqrt(mean_squared_error(y_train, y_pred))
sonuc_train_veri_detay_slr

305.50801973625977

In [1300]:
y_pred = reg_model.predict(X_test)

In [1301]:
sonuc_test_veri_detay_slr = np.sqrt(mean_squared_error(y_test, y_pred))
sonuc_test_veri_detay_slr

199.33479501511692

In [1302]:
sonuc_tt_veri_cv_detay_slr = np.sqrt(np.mean(-cross_val_score(reg_model, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error")))
sonuc_tt_veri_cv_detay_slr

307.5702326697859

## 4.2.1 Tüm veri ile çoklu doğrusal regresyon

In [1303]:
#Geri kalan tüm modellemelerde bu X, y, X_train ve y_train i  kullanacağız.
y = df["Salary"]
X = df.drop("Salary",axis=1)
X = scaler.fit_transform(X)
for i in range(len(X)):
    if X[i][13]<0:
        X[i][13]=0
    else:
        X[i][13]=1  
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=46)

In [1304]:
reg_model = LinearRegression()
reg_model.fit(X, y)

LinearRegression()

In [1305]:
reg_model.intercept_

532.2082007978919

In [1306]:
reg_model.coef_

array([ 106.51525871, -136.39694927, -307.85665416,   13.74256439,
        618.76364979,   11.78186749, -183.01879595, -311.67669992,
        542.18323431,   14.46507744, -209.53809786,   49.38456229,
        128.82584609,  -56.5630632 ])

In [1307]:
y_pred = reg_model.predict(X)

In [1308]:
sonuc_tum_veri_detay_clr = np.sqrt(mean_squared_error(y, y_pred))
sonuc_tum_veri_detay_clr

230.8966955241041

In [1309]:
sonuc_tum_veri_cv_detay_clr = np.sqrt(np.mean(-cross_val_score(reg_model, X, y, cv = 10, scoring = "neg_mean_squared_error")))
sonuc_tum_veri_cv_detay_clr

250.65626224156816

## 4.2.2 Train ve test ayırarak çoklu doğrusal regresyon

In [1310]:
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)

LinearRegression()

In [1311]:
reg_model.intercept_

540.8147190244803

In [1312]:
reg_model.coef_

array([  74.8093756 , -175.03632957, -296.9222592 ,  -21.23625085,
        625.40993564,   83.93366566, -154.25248501, -314.62554981,
        567.28992276,   46.77623013, -219.8671064 ,    2.16387392,
        121.23948802,  -70.55563517])

In [1313]:
y_pred = reg_model.predict(X_train)

In [1314]:
sonuc_train_veri_detay_clr = np.sqrt(mean_squared_error(y_train, y_pred))
sonuc_train_veri_detay_clr

242.23277533668204

In [1315]:
y_pred = reg_model.predict(X_test)

In [1316]:
sonuc_test_veri_detay_clr = np.sqrt(mean_squared_error(y_test, y_pred))
sonuc_test_veri_detay_clr

183.46682748847505

In [1317]:
sonuc_tt_veri_cv_detay_clr = np.sqrt(np.mean(-cross_val_score(reg_model, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error")))
sonuc_tt_veri_cv_detay_clr

263.14224262610526

## 4.3.1 Tüm veri ile Ridge Regresyon

In [1318]:
reg_model = Ridge()
reg_model.fit(X, y)

Ridge()

In [1319]:
reg_model.intercept_

532.9502763405726

In [1320]:
reg_model.coef_

array([  87.09740907, -153.87684513,   29.94348902,   87.20004419,
        310.83334174, -104.80454862, -115.68820993, -266.80292953,
        346.83658368,  -32.26628162,  -58.80654371,  112.98468324,
         93.46082373,  -58.01921144])

In [1321]:
y_pred = reg_model.predict(X)

In [1322]:
sonuc_tum_veri_detay_ridge = np.sqrt(mean_squared_error(y, y_pred))
sonuc_tum_veri_detay_ridge

231.98300777642373

In [1323]:
sonuc_tum_veri_cv_detay_ridge = np.sqrt(np.mean(-cross_val_score(reg_model, X, y, cv = 10, scoring = "neg_mean_squared_error")))
sonuc_tum_veri_cv_detay_ridge

249.75330617912172

## 4.3.2 Train ve test ayırarak Ridge Regresyon

In [1324]:
reg_model = Ridge()
reg_model.fit(X_train, y_train)

Ridge()

In [1325]:
reg_model.intercept_

541.5560825114876

In [1326]:
reg_model.coef_

array([  54.24754018, -162.11867482,   48.21645672,   71.89180512,
        294.9005973 ,  -69.83130998,  -82.41572932, -278.11121771,
        355.2194913 ,  -16.63123642,  -51.13878496,   91.53362313,
         81.68820653,  -73.07711113])

In [1327]:
y_pred = reg_model.predict(X_train)

In [1328]:
sonuc_train_veri_detay_ridge = np.sqrt(mean_squared_error(y_train, y_pred))
sonuc_train_veri_detay_ridge

243.55922491050313

In [1329]:
y_pred = reg_model.predict(X_test)

In [1330]:
sonuc_test_veri_detay_ridge = np.sqrt(mean_squared_error(y_test, y_pred))
sonuc_test_veri_detay_ridge

184.04577735421122

In [1331]:
sonuc_tt_veri_cv_detay_ridge = np.sqrt(np.mean(-cross_val_score(reg_model, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error")))
sonuc_tt_veri_cv_detay_ridge

259.4881504974487

## 4.4.1 Tüm veri ile Lasso Regresyon

In [1332]:
reg_model = Lasso()
reg_model.fit(X, y)

Lasso()

In [1333]:
reg_model.intercept_

532.5060271898013

In [1334]:
reg_model.coef_

array([  63.51176058,  -80.77517787,   -0.        ,   31.91999479,
        210.46990816,   -0.        ,  -82.43766624, -299.94419996,
        361.68728958,   -0.        ,   -5.52194309,   62.05718523,
         75.41119302,  -57.14747725])

In [1335]:
y_pred = reg_model.predict(X)

In [1336]:
sonuc_tum_veri_detay_lasso = np.sqrt(mean_squared_error(y, y_pred))
sonuc_tum_veri_detay_lasso

233.36978363520865

In [1337]:
sonuc_tum_veri_cv_detay_lasso = np.sqrt(np.mean(-cross_val_score(reg_model, X, y, cv = 10, scoring = "neg_mean_squared_error")))
sonuc_tum_veri_cv_detay_lasso

250.14002517167236

## 4.4.2 Train ve test ayırarak Lasso Regresyon

In [1338]:
reg_model = Lasso()
reg_model.fit(X_train, y_train)

Lasso()

In [1339]:
reg_model.intercept_

540.8378322342253

In [1340]:
reg_model.coef_

array([  34.02462811,  -87.03319124,   -0.        ,   36.24439389,
        228.8311611 ,   -0.        ,  -55.04577573, -326.1514941 ,
        387.26881295,    0.        ,  -11.10782921,   62.83264322,
         65.59750522,  -71.69281394])

In [1341]:
y_pred = reg_model.predict(X_train)

In [1342]:
sonuc_train_veri_detay_lasso = np.sqrt(mean_squared_error(y_train, y_pred))
sonuc_train_veri_detay_lasso

244.2926611468897

In [1343]:
y_pred = reg_model.predict(X_test)

In [1344]:
sonuc_test_veri_detay_lasso = np.sqrt(mean_squared_error(y_test, y_pred))
sonuc_test_veri_detay_lasso

186.99736011981014

In [1345]:
sonuc_tt_veri_cv_detay_lasso = np.sqrt(np.mean(-cross_val_score(reg_model, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error")))
sonuc_tt_veri_cv_detay_lasso

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4416.222686810419, tolerance: 3318.8661693549875
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3892.5810073222965, tolerance: 3635.704967965367
  positive)


259.9764808122278

## 4.5.1 Tüm veri ile ElasticNet Regresyon

In [1346]:
reg_model = ElasticNet()
reg_model.fit(X, y)

ElasticNet()

In [1347]:
reg_model.intercept_

516.7200312286989

In [1348]:
reg_model.coef_

array([ 21.14914281,  18.28663001,  30.84007789,  20.05589336,
        34.62793441,  24.31228884,  10.01743457,  12.35605737,
        41.69860685,  12.73213634,  34.11871793,  33.84861899,
        21.93565765, -26.17118329])

In [1349]:
y_pred = reg_model.predict(X)

In [1350]:
sonuc_tum_veri_detay_enet = np.sqrt(mean_squared_error(y, y_pred))
sonuc_tum_veri_detay_enet

250.31298370586887

In [1351]:
sonuc_tum_veri_cv_detay_enet = np.sqrt(np.mean(-cross_val_score(reg_model, X, y, cv = 10, scoring = "neg_mean_squared_error")))
sonuc_tum_veri_cv_detay_enet

257.430624312021

## 4.5.2 Train ve test ayırarak ElasticNet Regresyon

In [1352]:
reg_model = ElasticNet()
reg_model.fit(X_train, y_train)

ElasticNet()

In [1353]:
reg_model.intercept_

522.5645571930195

In [1354]:
reg_model.coef_

array([ 17.1065229 ,  17.10886879,  32.11395115,  22.59747031,
        37.53662701,  25.84971294,  13.9282492 ,   7.55859356,
        41.8910954 ,  14.23594283,  34.35565384,  33.30462076,
        25.13862855, -33.33576445])

In [1355]:
y_pred = reg_model.predict(X_train)

In [1356]:
sonuc_train_veri_detay_enet = np.sqrt(mean_squared_error(y_train, y_pred))
sonuc_train_veri_detay_enet

263.3465526580905

In [1357]:
y_pred = reg_model.predict(X_test)

In [1358]:
sonuc_test_veri_detay_enet = np.sqrt(mean_squared_error(y_test, y_pred))
sonuc_test_veri_detay_enet

186.2553747469711

In [1359]:
sonuc_tt_veri_cv_detay_enet = np.sqrt(np.mean(-cross_val_score(reg_model, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error")))
sonuc_tt_veri_cv_detay_enet

269.7061255415075

In [1360]:
#Detaylı sonuçların çıktısı
detaysonuc_df = pd.DataFrame({"Tüm Veri Score":[sonuc_tum_veri_detay_slr,sonuc_tum_veri_detay_clr,sonuc_tum_veri_detay_ridge,sonuc_tum_veri_detay_lasso,sonuc_tum_veri_detay_enet],
                              "Tüm Veri CVScore":[sonuc_tum_veri_cv_detay_slr,sonuc_tum_veri_cv_detay_clr,sonuc_tum_veri_cv_detay_ridge,sonuc_tum_veri_cv_detay_lasso,sonuc_tum_veri_cv_detay_enet],
                              "Train Veri Sonuc":[sonuc_train_veri_detay_slr,sonuc_train_veri_detay_clr,sonuc_train_veri_detay_ridge,sonuc_train_veri_detay_lasso,sonuc_train_veri_detay_enet],
                              "Test Veri Sonuc":[sonuc_test_veri_detay_slr,sonuc_test_veri_detay_clr,sonuc_test_veri_detay_ridge,sonuc_test_veri_detay_lasso,sonuc_test_veri_detay_enet],                              
                              "TrainTest CVScore":[sonuc_tt_veri_cv_detay_slr,sonuc_tt_veri_cv_detay_clr,sonuc_tt_veri_cv_detay_ridge,sonuc_tt_veri_cv_detay_lasso,sonuc_tt_veri_cv_detay_enet]})
detaysonuc_df.index= ["SLR_DETAY", "CLR_DETAY","RID_DETAY","LAS_DETAY","ENE_DETAY"]
detaysonuc_df

,Tüm Veri Score,Tüm Veri CVScore,Train Veri Sonuc,Test Veri Sonuc,TrainTest CVScore
SLR_DETAY,287.204586,290.057726,305.508020,199.334795,307.570233
CLR_DETAY,230.896696,250.656262,242.232775,183.466827,263.142243
RID_DETAY,231.983008,249.753306,243.559225,184.045777,259.488150
LAS_DETAY,233.369784,250.140025,244.292661,186.997360,259.976481
ENE_DETAY,250.312984,257.430624,263.346553,186.255375,269.706126


## 5. CV işlemleri Ridge, Lasso, Enet

In [1361]:
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV

# 5.1 RidgeCV

In [1362]:
alphas1 = np.random.randint(0,1000,100)
alphas2 = 10**np.linspace(10,-2,100)*0.5
alphas3 = np.linspace(0,1,1000)

In [1363]:
ridge_cv = RidgeCV(alphas = alphas1, scoring = "neg_mean_squared_error", cv = 10, normalize = True)

In [1364]:
ridge_cv.fit(X, y)

RidgeCV(alphas=array([669, 137,  32, 217, 100, 489, 792, 313, 343, 415,  71,  10, 446,
       503, 724, 980, 959, 654, 158, 846, 375, 276, 259, 905, 288, 375,
       623, 564, 414, 901, 575, 589, 679, 619, 306, 271, 355,  95, 948,
       280, 152, 293, 642, 607, 220, 119, 802, 293, 197,  38, 435, 261,
       431, 388, 494, 571, 352, 328, 175, 842, 604,  74, 664, 334, 873,
        74, 179, 598, 675, 372, 713, 350, 594, 350, 144, 397, 101, 662,
       286, 644, 357, 139, 576, 313, 280, 463, 552, 441, 493, 647, 410,
        94, 152, 472, 129, 105, 452, 794, 769, 476]),
        cv=10, normalize=True, scoring='neg_mean_squared_error')

In [1365]:
tum_alpha_ridge1 = ridge_cv.alpha_
tum_alpha_ridge1

10

In [1366]:
ridge_cv.fit(X_train, y_train)

RidgeCV(alphas=array([669, 137,  32, 217, 100, 489, 792, 313, 343, 415,  71,  10, 446,
       503, 724, 980, 959, 654, 158, 846, 375, 276, 259, 905, 288, 375,
       623, 564, 414, 901, 575, 589, 679, 619, 306, 271, 355,  95, 948,
       280, 152, 293, 642, 607, 220, 119, 802, 293, 197,  38, 435, 261,
       431, 388, 494, 571, 352, 328, 175, 842, 604,  74, 664, 334, 873,
        74, 179, 598, 675, 372, 713, 350, 594, 350, 144, 397, 101, 662,
       286, 644, 357, 139, 576, 313, 280, 463, 552, 441, 493, 647, 410,
        94, 152, 472, 129, 105, 452, 794, 769, 476]),
        cv=10, normalize=True, scoring='neg_mean_squared_error')

In [1367]:
tt_alpha_ridge1 = ridge_cv.alpha_
tt_alpha_ridge1 

10

In [1368]:
ridge_tuned = Ridge(alpha = tum_alpha_ridge1).fit(X, y)
y_pred = ridge_tuned.predict(X)
tum_tuned_ridge1 = np.sqrt(mean_squared_error(y, y_pred))
tum_tuned_ridge1

238.9462636562976

In [1369]:
ridge_tuned = Ridge(alpha = tt_alpha_ridge1 ).fit(X_train, y_train)
y_pred = ridge_tuned.predict(X_test)
tt_tuned_ridge1 = np.sqrt(mean_squared_error(y_test, y_pred))
tt_tuned_ridge1

186.81369979884866

In [1370]:
ridge_cv = RidgeCV(alphas = alphas2, scoring = "neg_mean_squared_error", cv = 10, normalize = True)

In [1371]:
ridge_cv.fit(X, y)

RidgeCV(alphas=array([5.00000000e+09, 3.78231664e+09, 2.86118383e+09, 2.16438064e+09,
       1.63727458e+09, 1.23853818e+09, 9.36908711e+08, 7.08737081e+08,
       5.36133611e+08, 4.05565415e+08, 3.06795364e+08, 2.32079442e+08,
       1.75559587e+08, 1.32804389e+08, 1.00461650e+08, 7.59955541e+07,
       5.74878498e+07, 4.34874501e+07, 3.28966612e+07, 2.48851178e+07,
       1.88246790e+07, 1.42401793e+0...
       1.00461650e+00, 7.59955541e-01, 5.74878498e-01, 4.34874501e-01,
       3.28966612e-01, 2.48851178e-01, 1.88246790e-01, 1.42401793e-01,
       1.07721735e-01, 8.14875417e-02, 6.16423370e-02, 4.66301673e-02,
       3.52740116e-02, 2.66834962e-02, 2.01850863e-02, 1.52692775e-02,
       1.15506485e-02, 8.73764200e-03, 6.60970574e-03, 5.00000000e-03]),
        cv=10, normalize=True, scoring='neg_mean_squared_error')

In [1372]:
tum_alpha_ridge2 = ridge_cv.alpha_
tum_alpha_ridge2

0.005

In [1373]:
ridge_cv.fit(X_train, y_train)

RidgeCV(alphas=array([5.00000000e+09, 3.78231664e+09, 2.86118383e+09, 2.16438064e+09,
       1.63727458e+09, 1.23853818e+09, 9.36908711e+08, 7.08737081e+08,
       5.36133611e+08, 4.05565415e+08, 3.06795364e+08, 2.32079442e+08,
       1.75559587e+08, 1.32804389e+08, 1.00461650e+08, 7.59955541e+07,
       5.74878498e+07, 4.34874501e+07, 3.28966612e+07, 2.48851178e+07,
       1.88246790e+07, 1.42401793e+0...
       1.00461650e+00, 7.59955541e-01, 5.74878498e-01, 4.34874501e-01,
       3.28966612e-01, 2.48851178e-01, 1.88246790e-01, 1.42401793e-01,
       1.07721735e-01, 8.14875417e-02, 6.16423370e-02, 4.66301673e-02,
       3.52740116e-02, 2.66834962e-02, 2.01850863e-02, 1.52692775e-02,
       1.15506485e-02, 8.73764200e-03, 6.60970574e-03, 5.00000000e-03]),
        cv=10, normalize=True, scoring='neg_mean_squared_error')

In [1374]:
tt_alpha_ridge2 = ridge_cv.alpha_
tt_alpha_ridge2

0.005

In [1375]:
ridge_tuned = Ridge(alpha = tum_alpha_ridge2).fit(X, y)
y_pred = ridge_tuned.predict(X)
tum_tuned_ridge2 = np.sqrt(mean_squared_error(y, y_pred))
tum_tuned_ridge2

230.89706045855954

In [1376]:
ridge_tuned = Ridge(alpha = tt_alpha_ridge2).fit(X_train, y_train)
y_pred = ridge_tuned.predict(X_test)
tt_tuned_ridge2 = np.sqrt(mean_squared_error(y_test, y_pred))
tt_tuned_ridge2

183.45697041486676

In [1377]:
ridge_cv = RidgeCV(alphas = alphas3, scoring = "neg_mean_squared_error", cv = 10, normalize = True)

In [1378]:
ridge_cv.fit(X, y)

RidgeCV(alphas=array([0.        , 0.001001  , 0.002002  , 0.003003  , 0.004004  ,
       0.00500501, 0.00600601, 0.00700701, 0.00800801, 0.00900901,
       0.01001001, 0.01101101, 0.01201201, 0.01301301, 0.01401401,
       0.01501502, 0.01601602, 0.01701702, 0.01801802, 0.01901902,
       0.02002002, 0.02102102, 0.02202202, 0.02302302, 0.02402402,
       0.02502503, 0.02602603, 0.02702703, 0.02802803, 0.02902903,
       0.03003003, 0.0...
       0.97097097, 0.97197197, 0.97297297, 0.97397397, 0.97497497,
       0.97597598, 0.97697698, 0.97797798, 0.97897898, 0.97997998,
       0.98098098, 0.98198198, 0.98298298, 0.98398398, 0.98498498,
       0.98598599, 0.98698699, 0.98798799, 0.98898899, 0.98998999,
       0.99099099, 0.99199199, 0.99299299, 0.99399399, 0.99499499,
       0.995996  , 0.996997  , 0.997998  , 0.998999  , 1.        ]),
        cv=10, normalize=True, scoring='neg_mean_squared_error')

In [1379]:
tum_alpha_ridge3 = ridge_cv.alpha_
tum_alpha_ridge3

0.002002002002002002

In [1380]:
ridge_cv.fit(X_train, y_train)

RidgeCV(alphas=array([0.        , 0.001001  , 0.002002  , 0.003003  , 0.004004  ,
       0.00500501, 0.00600601, 0.00700701, 0.00800801, 0.00900901,
       0.01001001, 0.01101101, 0.01201201, 0.01301301, 0.01401401,
       0.01501502, 0.01601602, 0.01701702, 0.01801802, 0.01901902,
       0.02002002, 0.02102102, 0.02202202, 0.02302302, 0.02402402,
       0.02502503, 0.02602603, 0.02702703, 0.02802803, 0.02902903,
       0.03003003, 0.0...
       0.97097097, 0.97197197, 0.97297297, 0.97397397, 0.97497497,
       0.97597598, 0.97697698, 0.97797798, 0.97897898, 0.97997998,
       0.98098098, 0.98198198, 0.98298298, 0.98398398, 0.98498498,
       0.98598599, 0.98698699, 0.98798799, 0.98898899, 0.98998999,
       0.99099099, 0.99199199, 0.99299299, 0.99399399, 0.99499499,
       0.995996  , 0.996997  , 0.997998  , 0.998999  , 1.        ]),
        cv=10, normalize=True, scoring='neg_mean_squared_error')

In [1381]:
tt_alpha_ridge3 = ridge_cv.alpha_
tt_alpha_ridge3

0.005005005005005005

In [1382]:
ridge_tuned = Ridge(alpha = tum_alpha_ridge3).fit(X, y)
y_pred = ridge_tuned.predict(X)
tum_tuned_ridge3 = np.sqrt(mean_squared_error(y, y_pred))
tum_tuned_ridge3

230.89675582253818

In [1383]:
ridge_tuned = Ridge(alpha = tt_alpha_ridge3).fit(X_train, y_train)
y_pred = ridge_tuned.predict(X_test)
tt_tuned_ridge3 = np.sqrt(mean_squared_error(y_test, y_pred))
tt_tuned_ridge3

183.45696108934007

In [1384]:
if (tum_tuned_ridge1 < tum_tuned_ridge2) & (tum_tuned_ridge1 < tum_tuned_ridge3):
    tum_tuned_ridge = tum_tuned_ridge1
    tum_alpha_ridge = tum_alpha_ridge1
elif tum_tuned_ridge2 < tum_tuned_ridge3:
    tum_tuned_ridge = tum_tuned_ridge2
    tum_alpha_ridge = tum_alpha_ridge2
else:
    tum_tuned_ridge = tum_tuned_ridge3
    tum_alpha_ridge = tum_alpha_ridge3
print("RMSE:"+str(tum_tuned_ridge)+"  Alpha:"+str(tum_alpha_ridge))

RMSE:230.89675582253818  Alpha:0.002002002002002002


In [1385]:
if (tt_tuned_ridge1 < tt_tuned_ridge2) & (tt_tuned_ridge1 < tt_tuned_ridge3):
    tt_tuned_ridge = tt_tuned_ridge1
    tt_alpha_ridge = tt_alpha_ridge1
elif tum_tuned_ridge2 < tum_tuned_ridge3:
    tt_tuned_ridge = tt_tuned_ridge2
    tt_alpha_ridge = tt_alpha_ridge2
else:
    tt_tuned_ridge = tt_tuned_ridge3
    tt_alpha_ridge = tt_alpha_ridge3
print("RMSE:"+str(tt_tuned_ridge)+"  Alpha:"+str(tt_alpha_ridge))

RMSE:183.45696108934007  Alpha:0.005005005005005005


# 5.2 LassoCv

In [1386]:
alphas1 = np.random.randint(0,1000,100)
alphas2 = 10**np.linspace(10,-2,100)*0.5
alphas3 = np.linspace(0,1,1000)

In [1387]:
lasso_cv = LassoCV(alphas = alphas1, cv = 10)

In [1388]:
lasso_cv.fit(X, y)

LassoCV(alphas=array([481, 574, 336, 255, 573, 520, 330, 980, 915,  30,  11, 902, 788,
       703, 569, 227, 374, 725, 491, 168, 729,  14, 534,  18, 898,  79,
       626, 733, 450, 378, 412, 317, 583, 660, 688, 888, 862, 979, 371,
       156, 153, 413, 212, 752, 834, 914, 772, 121, 665, 680, 428, 937,
       211, 751, 956, 255, 431,  58, 866, 521, 678, 607, 170, 361, 895,
       151, 323, 610, 511,  92, 525, 323, 690, 745, 797, 236, 566, 549,
       461, 847, 586, 421, 205, 535, 131, 770, 672, 957,  52, 406, 350,
         8, 154, 675, 690, 229, 871, 586, 511, 502]),
        cv=10)

In [1389]:
tum_alpha_lasso1 = lasso_cv.alpha_
tum_alpha_lasso1

8

In [1390]:
lasso_cv.fit(X_train, y_train)

LassoCV(alphas=array([481, 574, 336, 255, 573, 520, 330, 980, 915,  30,  11, 902, 788,
       703, 569, 227, 374, 725, 491, 168, 729,  14, 534,  18, 898,  79,
       626, 733, 450, 378, 412, 317, 583, 660, 688, 888, 862, 979, 371,
       156, 153, 413, 212, 752, 834, 914, 772, 121, 665, 680, 428, 937,
       211, 751, 956, 255, 431,  58, 866, 521, 678, 607, 170, 361, 895,
       151, 323, 610, 511,  92, 525, 323, 690, 745, 797, 236, 566, 549,
       461, 847, 586, 421, 205, 535, 131, 770, 672, 957,  52, 406, 350,
         8, 154, 675, 690, 229, 871, 586, 511, 502]),
        cv=10)

In [1391]:
tt_alpha_lasso1 = lasso_cv.alpha_
tt_alpha_lasso1

8

In [1392]:
lasso_tuned = Lasso(alpha = tum_alpha_lasso1).fit(X, y)
y_pred = lasso_tuned.predict(X)
tum_tuned_lasso1 = np.sqrt(mean_squared_error(y, y_pred))
tum_tuned_lasso1

246.2242671391851

In [1393]:
lasso_tuned = Lasso(alpha = tt_alpha_lasso1).fit(X_train, y_train)
y_pred = lasso_tuned.predict(X_test)
tt_tuned_lasso1 = np.sqrt(mean_squared_error(y_test, y_pred))
tt_tuned_lasso1

187.3546623752127

In [1394]:
lasso_cv = LassoCV(alphas = alphas2, cv = 10)

In [1395]:
lasso_cv.fit(X, y)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18196.75665498525, tolerance: 4193.070265946953
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 66264.17131484859, tolerance: 4193.070265946953
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 82653.91548405215, tolerance: 4193.070265946953
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increa

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 15033.981579776853, tolerance: 4227.851227760603
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18774.735487181693, tolerance: 4227.851227760603
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 33240.96821513958, tolerance: 4227.851227760603
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to incr

LassoCV(alphas=array([5.00000000e+09, 3.78231664e+09, 2.86118383e+09, 2.16438064e+09,
       1.63727458e+09, 1.23853818e+09, 9.36908711e+08, 7.08737081e+08,
       5.36133611e+08, 4.05565415e+08, 3.06795364e+08, 2.32079442e+08,
       1.75559587e+08, 1.32804389e+08, 1.00461650e+08, 7.59955541e+07,
       5.74878498e+07, 4.34874501e+07, 3.28966612e+07, 2.48851178e+07,
       1.88246790e+07, 1.42401793e+0...
       3.06795364e+00, 2.32079442e+00, 1.75559587e+00, 1.32804389e+00,
       1.00461650e+00, 7.59955541e-01, 5.74878498e-01, 4.34874501e-01,
       3.28966612e-01, 2.48851178e-01, 1.88246790e-01, 1.42401793e-01,
       1.07721735e-01, 8.14875417e-02, 6.16423370e-02, 4.66301673e-02,
       3.52740116e-02, 2.66834962e-02, 2.01850863e-02, 1.52692775e-02,
       1.15506485e-02, 8.73764200e-03, 6.60970574e-03, 5.00000000e-03]),
        cv=10)

In [1396]:
tum_alpha_lasso2 = lasso_cv.alpha_
tum_alpha_lasso2

1.328043891473342

In [1397]:
lasso_cv.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4133.975385064259, tolerance: 3318.8661693549875
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4807.822110753506, tolerance: 3318.8661693549875
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4574.87047075294, tolerance: 3318.8661693549875
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to incr

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7192.334211129695, tolerance: 3624.3492833112487
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6832.086621271446, tolerance: 3624.3492833112487
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5232.184050172567, tolerance: 3624.3492833112487
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to inc

LassoCV(alphas=array([5.00000000e+09, 3.78231664e+09, 2.86118383e+09, 2.16438064e+09,
       1.63727458e+09, 1.23853818e+09, 9.36908711e+08, 7.08737081e+08,
       5.36133611e+08, 4.05565415e+08, 3.06795364e+08, 2.32079442e+08,
       1.75559587e+08, 1.32804389e+08, 1.00461650e+08, 7.59955541e+07,
       5.74878498e+07, 4.34874501e+07, 3.28966612e+07, 2.48851178e+07,
       1.88246790e+07, 1.42401793e+0...
       3.06795364e+00, 2.32079442e+00, 1.75559587e+00, 1.32804389e+00,
       1.00461650e+00, 7.59955541e-01, 5.74878498e-01, 4.34874501e-01,
       3.28966612e-01, 2.48851178e-01, 1.88246790e-01, 1.42401793e-01,
       1.07721735e-01, 8.14875417e-02, 6.16423370e-02, 4.66301673e-02,
       3.52740116e-02, 2.66834962e-02, 2.01850863e-02, 1.52692775e-02,
       1.15506485e-02, 8.73764200e-03, 6.60970574e-03, 5.00000000e-03]),
        cv=10)

In [1398]:
tt_alpha_lasso2 = lasso_cv.alpha_
tt_alpha_lasso2

1.328043891473342

In [1399]:
lasso_tuned = Lasso(alpha = tum_alpha_lasso2).fit(X, y)
y_pred = lasso_tuned.predict(X)
tum_tuned_lasso2 = np.sqrt(mean_squared_error(y, y_pred))
tum_tuned_lasso2

233.95492954649598

In [1400]:
lasso_tuned = Lasso(alpha = tt_alpha_lasso2).fit(X_train, y_train)
y_pred = lasso_tuned.predict(X_test)
tt_tuned_lasso2 = np.sqrt(mean_squared_error(y_test, y_pred))
tt_tuned_lasso2

187.67602939446942

In [1401]:
lasso_cv = LassoCV(alphas = alphas3, cv = 10)

In [1402]:
lasso_cv.fit(X, y)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4358.831252578646, tolerance: 4193.070265946953
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4897.85869994387, tolerance: 4193.070265946953
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5650.0258618704975, tolerance: 4193.070265946953
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increa

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4009.6780343540013, tolerance: 3931.2824868948537
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4204.459961570799, tolerance: 3931.2824868948537
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4450.192939672619, tolerance: 3931.2824868948537
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to in

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4531.227547170594, tolerance: 4119.588748451991
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4707.358335915953, tolerance: 4119.588748451991
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4884.668894810602, tolerance: 4119.588748451991
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increa

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4109.15847947076, tolerance: 3842.888948307051
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4357.016138069332, tolerance: 3842.888948307051
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4639.651246998459, tolerance: 3842.888948307051
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increas

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4417.099724471569, tolerance: 4227.851227760603
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4904.6589713376015, tolerance: 4227.851227760603
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5552.798163022846, tolerance: 4227.851227760603
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to incre

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7508.302592530847, tolerance: 4047.6092280925
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7186.34323505871, tolerance: 4047.6092280925
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7709.015957634896, tolerance: 4047.6092280925
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the 

LassoCV(alphas=array([0.        , 0.001001  , 0.002002  , 0.003003  , 0.004004  ,
       0.00500501, 0.00600601, 0.00700701, 0.00800801, 0.00900901,
       0.01001001, 0.01101101, 0.01201201, 0.01301301, 0.01401401,
       0.01501502, 0.01601602, 0.01701702, 0.01801802, 0.01901902,
       0.02002002, 0.02102102, 0.02202202, 0.02302302, 0.02402402,
       0.02502503, 0.02602603, 0.02702703, 0.02802803, 0.02902903,
       0.03003003, 0.0...
       0.96596597, 0.96696697, 0.96796797, 0.96896897, 0.96996997,
       0.97097097, 0.97197197, 0.97297297, 0.97397397, 0.97497497,
       0.97597598, 0.97697698, 0.97797798, 0.97897898, 0.97997998,
       0.98098098, 0.98198198, 0.98298298, 0.98398398, 0.98498498,
       0.98598599, 0.98698699, 0.98798799, 0.98898899, 0.98998999,
       0.99099099, 0.99199199, 0.99299299, 0.99399399, 0.99499499,
       0.995996  , 0.996997  , 0.997998  , 0.998999  , 1.        ]),
        cv=10)

In [1403]:
tum_alpha_lasso3 = lasso_cv.alpha_
tum_alpha_lasso3

0.7647647647647647

In [1404]:
lasso_cv.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4416.2226870544255, tolerance: 3318.8661693549875
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4196.8492963165045, tolerance: 3318.8661693549875
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5845.837914992124, tolerance: 3318.8661693549875
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to i

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3775.600187269971, tolerance: 3553.4109035894603
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3944.7584057971835, tolerance: 3553.4109035894603
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4113.526884512976, tolerance: 3553.4109035894603
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to in

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3951.055763363838, tolerance: 3636.654275064953
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4606.466675229371, tolerance: 3636.654275064953
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5500.697217449546, tolerance: 3636.654275064953
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increa

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3942.7957511767745, tolerance: 3635.704967965367
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4117.849679231644, tolerance: 3635.704967965367
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4272.113175623119, tolerance: 3635.704967965367
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to incre

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4161.787768408656, tolerance: 3624.3492833112487
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5116.7081092484295, tolerance: 3624.3492833112487
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5768.587981905788, tolerance: 3624.3492833112487
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to in

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4019.8965668305755, tolerance: 3613.8783061157274
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4853.184700835496, tolerance: 3613.8783061157274
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6061.3687854688615, tolerance: 3613.8783061157274
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to i

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3756.0507325306535, tolerance: 3532.1625860921995
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4235.013918239623, tolerance: 3532.1625860921995
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4860.932534523308, tolerance: 3532.1625860921995
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to in

LassoCV(alphas=array([0.        , 0.001001  , 0.002002  , 0.003003  , 0.004004  ,
       0.00500501, 0.00600601, 0.00700701, 0.00800801, 0.00900901,
       0.01001001, 0.01101101, 0.01201201, 0.01301301, 0.01401401,
       0.01501502, 0.01601602, 0.01701702, 0.01801802, 0.01901902,
       0.02002002, 0.02102102, 0.02202202, 0.02302302, 0.02402402,
       0.02502503, 0.02602603, 0.02702703, 0.02802803, 0.02902903,
       0.03003003, 0.0...
       0.96596597, 0.96696697, 0.96796797, 0.96896897, 0.96996997,
       0.97097097, 0.97197197, 0.97297297, 0.97397397, 0.97497497,
       0.97597598, 0.97697698, 0.97797798, 0.97897898, 0.97997998,
       0.98098098, 0.98198198, 0.98298298, 0.98398398, 0.98498498,
       0.98598599, 0.98698699, 0.98798799, 0.98898899, 0.98998999,
       0.99099099, 0.99199199, 0.99299299, 0.99399399, 0.99499499,
       0.995996  , 0.996997  , 0.997998  , 0.998999  , 1.        ]),
        cv=10)

In [1405]:
tt_alpha_lasso3 = lasso_cv.alpha_
tt_alpha_lasso3

1.0

In [1406]:
lasso_tuned = Lasso(alpha = tt_alpha_lasso3).fit(X, y)
y_pred = lasso_tuned.predict(X)
tum_tuned_lasso3 = np.sqrt(mean_squared_error(y, y_pred))
tum_tuned_lasso3

233.36978363520865

In [1407]:
lasso_tuned = Lasso(alpha = tt_alpha_lasso3).fit(X_train, y_train)
y_pred = lasso_tuned.predict(X_test)
tt_tuned_lasso3 = np.sqrt(mean_squared_error(y_test, y_pred))
tt_tuned_lasso3

186.99736011981014

In [1408]:
if (tum_tuned_lasso1 < tum_tuned_lasso2) & (tum_tuned_lasso1 < tum_tuned_lasso3):
    tum_tuned_lasso = tum_tuned_lasso1
    tum_alpha_lasso = tum_alpha_lasso1
elif tum_tuned_lasso2 < tum_tuned_lasso3:
    tum_tuned_lasso = tum_tuned_lasso2
    tum_alpha_lasso = tum_alpha_lasso2
else:
    tum_tuned_lasso = tum_tuned_lasso3
    tum_alpha_lasso = tum_alpha_lasso3
print("RMSE:"+str(tum_tuned_lasso)+"  Alpha:"+str(tum_alpha_lasso))

RMSE:233.36978363520865  Alpha:0.7647647647647647


In [1409]:
if (tt_tuned_lasso1 < tt_tuned_lasso2) & (tt_tuned_lasso1 < tt_tuned_lasso3):
    tt_tuned_lasso = tt_tuned_lasso1
    tt_alpha_lasso = tt_alpha_lasso1
elif tt_tuned_lasso2 < tt_tuned_lasso3:
    tt_tuned_lasso = tt_tuned_lasso2
    tt_alpha_lasso = tt_alpha_lasso2
else:
    tt_tuned_lasso = tt_tuned_lasso3
    tt_alpha_lasso = tt_alpha_lasso3
print("RMSE:"+str(tt_tuned_lasso)+"  Alpha:"+str(tt_alpha_lasso))

RMSE:186.99736011981014  Alpha:1.0


# 5.3 EnetCV

In [1410]:
from sklearn.model_selection import GridSearchCV
enet_params = {"l1_ratio": [0.1,0.2,0.4,0.5,0.6,0.8,1],
              "alpha":[0.1,0.01,0.001,0.2,0.3,0.5,0.8,0.9,1]}
enet_model = ElasticNet()

In [1411]:
gs_cv_enet = GridSearchCV(enet_model, enet_params, cv = 10).fit(X, y)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2662315.2804394383, tolerance: 4193.070265946953
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3498639.3417188525, tolerance: 4099.947434245417
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2887986.8129214086, tolerance: 3931.2824868948537
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1878530.

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 31926.37411880307, tolerance: 3903.4699647744806
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 173003.32816809043, tolerance: 4119.588748451991
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 249919.00073890015, tolerance: 3842.888948307051
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 337228.71

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5014370.299109183, tolerance: 4103.199871983576
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4279409.529472025, tolerance: 4047.6092280925
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6924108.867700958, tolerance: 4193.070265946953
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7516089.319650

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7304117.683659219, tolerance: 4119.588748451991
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7417068.252321411, tolerance: 3842.888948307051
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7707043.500673181, tolerance: 4227.851227760603
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7295964.9135

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7216443.107449035, tolerance: 4047.6092280925
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7324655.653280681, tolerance: 4193.070265946953
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7880436.318277209, tolerance: 4099.947434245417
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7150629.764461

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2196642.9626009567, tolerance: 4504.007527899855
  positive)


In [1412]:
gs_cv_enet.best_params_

{'alpha': 0.01, 'l1_ratio': 0.6}

In [1413]:
tum_tuned_enet_a = gs_cv_enet.best_params_["alpha"]
tum_tuned_enet_a

0.01

In [1414]:
tum_tuned_enet_l = gs_cv_enet.best_params_["l1_ratio"]
tum_tuned_enet_l

0.6

In [1415]:
enet_tuned = ElasticNet(**gs_cv_enet.best_params_).fit(X, y)
y_pred = enet_tuned.predict(X)
tum_tuned_enet = np.sqrt(mean_squared_error(y, y_pred))
tum_tuned_enet 

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2196642.9626009567, tolerance: 4504.007527899855
  positive)


232.28845595648767

In [1416]:
gs_cv_enet = GridSearchCV(enet_model, enet_params, cv = 10).fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1637782.9819639837, tolerance: 3318.8661693549875
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3191513.3682060405, tolerance: 3379.102918664405
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2867648.0027305214, tolerance: 3553.4109035894603
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2346489

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 398602.0377258919, tolerance: 3635.704967965367
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 72599.25220481306, tolerance: 3624.3492833112487
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 134231.68744674325, tolerance: 3613.8783061157274
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 79715.226

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6419483.983310111, tolerance: 3636.654275064953
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6507917.490736397, tolerance: 3568.5528744289877
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6466474.853603738, tolerance: 3635.704967965367
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5818551.868

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6497042.094587048, tolerance: 3553.4109035894603
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6803244.286193036, tolerance: 3636.654275064953
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6828895.372273407, tolerance: 3568.5528744289877
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6824725.97

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 636478.9854168519, tolerance: 3318.8661693549875
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 849547.4863362452, tolerance: 3379.102918664405
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 499452.31313841604, tolerance: 3553.4109035894603
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 344679.43

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4416.222686810419, tolerance: 3318.8661693549875
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3892.5810073222965, tolerance: 3635.704967965367
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2170647.2239074474, tolerance: 3935.871925203632
  positive)


In [1417]:
gs_cv_enet.best_params_

{'alpha': 0.01, 'l1_ratio': 0.6}

In [1418]:
tt_tuned_enet_a = gs_cv_enet.best_params_["alpha"]
tt_tuned_enet_a

0.01

In [1419]:
tt_tuned_enet_l = gs_cv_enet.best_params_["l1_ratio"]
tt_tuned_enet_l

0.6

In [1420]:
enet_tuned = ElasticNet(**gs_cv_enet.best_params_).fit(X_train, y_train)
y_pred = enet_tuned.predict(X_test)
tt_tuned_enet = np.sqrt(mean_squared_error(y_test, y_pred))
tt_tuned_enet 

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2170647.2239074474, tolerance: 3935.871925203632
  positive)


184.06570971410756

In [1421]:
#Detaylı sonuçların çıktısı
detaysonuc_df = pd.DataFrame({"All Data Error":[sonuc_tum_veri_detay_slr,sonuc_tum_veri_detay_clr,sonuc_tum_veri_detay_ridge,sonuc_tum_veri_detay_lasso,sonuc_tum_veri_detay_enet],
                              "All Data cvScore(K-Fold)":[sonuc_tum_veri_cv_detay_slr,sonuc_tum_veri_cv_detay_clr,sonuc_tum_veri_cv_detay_ridge,sonuc_tum_veri_cv_detay_lasso,sonuc_tum_veri_cv_detay_enet],
                              "All Data Tuned Error":[np.nan,np.nan,tum_tuned_ridge,tum_tuned_lasso,tum_tuned_enet],
                              "Train Error":[sonuc_train_veri_detay_slr,sonuc_train_veri_detay_clr,sonuc_train_veri_detay_ridge,sonuc_train_veri_detay_lasso,sonuc_train_veri_detay_enet],
                              "Test Error":[sonuc_test_veri_detay_slr,sonuc_test_veri_detay_clr,sonuc_test_veri_detay_ridge,sonuc_test_veri_detay_lasso,sonuc_test_veri_detay_enet],                              
                              "Train cvScore(K-Fold)":[sonuc_tt_veri_cv_detay_slr,sonuc_tt_veri_cv_detay_clr,sonuc_tt_veri_cv_detay_ridge,sonuc_tt_veri_cv_detay_lasso,sonuc_tt_veri_cv_detay_enet],
                              "Test Tuned Error":[np.nan,np.nan,tt_tuned_ridge,tt_tuned_lasso,tt_tuned_enet],
                              "Alphas":[np.nan,np.nan,tt_alpha_ridge,tt_alpha_lasso,tt_tuned_enet_a]})
detaysonuc_df.index= ["SLR_DETAY", "CLR_DETAY","RID_DETAY","LAS_DETAY","ENE_DETAY"]

In [1422]:
detaysonuc_df

,All Data Error,All Data cvScore(K-Fold),All Data Tuned Error,Train Error,Test Error,Train cvScore(K-Fold),Test Tuned Error,Alphas
SLR_DETAY,287.204586,290.057726,NaN,305.508020,199.334795,307.570233,NaN,NaN
CLR_DETAY,230.896696,250.656262,NaN,242.232775,183.466827,263.142243,NaN,NaN
RID_DETAY,231.983008,249.753306,230.896756,243.559225,184.045777,259.488150,183.456961,0.005005
LAS_DETAY,233.369784,250.140025,233.369784,244.292661,186.997360,259.976481,186.997360,1.000000
ENE_DETAY,250.312984,257.430624,232.288456,263.346553,186.255375,269.706126,184.065710,0.010000


In [1423]:
basicsonuc_df

,All Data Error,All Data cvScore(K-Fold),Train Error,Test Error,Train cvScore(K-Fold)
SLR,376.364303,383.452652,348.967844,472.230050,352.695909
CLR,303.344473,341.465977,261.361519,479.071504,307.225761
RID,304.114038,338.453184,264.576760,458.751443,301.308450
LAS,304.317294,338.626547,263.705645,461.242286,301.120525
ENE,326.562196,343.059463,287.380524,462.758530,302.314483
